<a href="https://colab.research.google.com/github/pranj-mach/git/blob/main/voice_text_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install speechrecognition transformers gtts

import speech_recognition as sr
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from gtts import gTTS
import IPython.display as display
import os

# Function to convert text to speech and play in Colab
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    display.display(display.Audio("response.mp3", autoplay=True))

# Transcribe speech from an uploaded audio file
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            return text
        except sr.UnknownValueError:
            return "Sorry, I couldn't understand."
        except sr.RequestError:
            return "API request error."

# Load emotion analysis model
tokenizer_emotion = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
emotion_model = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emotion")

def analyze_emotion(text):
    result = emotion_model(text)[0]
    return result["label"], result["score"]

# Load LLM model for response generation
llm_model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

def generate_response(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = llm_model.generate(**inputs, max_length=100, min_length=20, length_penalty=2.0, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Upload a WAV file in Colab
from google.colab import files

uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # Get the uploaded file name

# Process the uploaded file
user_text = transcribe_audio(file_path)
print(f"**You said:** {user_text}")

# Analyze emotion
emotion, confidence = analyze_emotion(user_text)
print(f"**Detected Emotion:** {emotion} (Confidence: {confidence:.2f})")

# Generate AI response
llm_response = generate_response(user_text)
print(f"**AI Analysis:** {llm_response}")

# Speak the response
response = f"I sense that you are feeling {emotion}. {llm_response}"
print(response)
speak(response)


Device set to use cuda:0


Saving The most essential t.wav to The most essential t.wav
**You said:** the most essential thing is to remain positive like you should not think what happened 5 years back or 7 years back or 10 years back because you will keep on regretting the thing we need to do is we should always see what we can do in the next 5 years because from 5 years from now we should not think okay what could happen like we should not feel any regret that 5 years ago I should have
**Detected Emotion:** anger (Confidence: 0.60)
**AI Analysis:**  The most essential thing is to remain positive like you should not think what happened 5 years back or 7 years back because you will keep on regretting . We should always see what we can do in the next 5 years because from 5 years from now we should not feel any regret that 5 years ago I should have .
I sense that you are feeling anger.  The most essential thing is to remain positive like you should not think what happened 5 years back or 7 years back because you wi